   Author: Ankit Kariryaa, University of Bremen

### Getting started
Define the paths to the dataset and trained models in the `notebooks/config/UNetTraining.py` file.  

In [7]:
import sys
print(sys.version)
print(sys.executable)
#sys.setdefaultencoding('utf-8')

3.7.16 (default, Jan 17 2023, 16:06:28) [MSC v.1916 64 bit (AMD64)]
D:\Software\anaconda\envs\acmtensorflow\python.exe


In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import imageio
import os
import time
import rasterio.warp             # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau


#from core.UNet import UNet
from core.UNetAttention import UNet  # upgrade UNet
from core.losses import tversky, accuracy, dice_coef, dice_loss, specificity, sensitivity
from core.losses_FTL import focalTversky,accuracy,dice_coef,dice_loss,true_positives,false_positives,true_negatives,false_negatives,sensitivity,specificity,PA,IoU_Pos,IoU_Neg,mIoU,F1_Score
from core.optimizers import adaDelta
# from tensorflow.keras.optimizers import legacy

from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.split_frames import split_dataset
from core.visualize import display_images

import json
from sklearn.model_selection import train_test_split

#%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon

import os


import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

#%reload_ext autoreload
#%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# ——————
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
# from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [2]:
# GPU
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

config1 = tf.compat.v1.ConfigProto(allow_soft_placement=False, log_device_placement=False)
config1.gpu_options.allow_growth = True
session=tf.compat.v1.Session(config=config1)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1197791086570611488
xla_global_id: -1
]
2.11.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

Num GPUs Available:  0


In [3]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/UNetTraining.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import UNetTraining
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder
# Eg. from configLargeCluster import UNetTraining
config = UNetTraining.Configuration()

In [4]:
frames = []

all_files = os.listdir(config.base_dir)
#all_files=all_files[:len(all_files)//2]
all_files_pan = [fn for fn in all_files if fn.startswith(config.pan_fn) and fn.endswith(config.image_type)]
for i, fn in enumerate(all_files_pan):
    pan_img = rasterio.open(os.path.join(config.base_dir, fn))
    #pan_img = rasterio.open(os.path.join(config.base_dir, fn.replace(config.ndvi_fn,config.pan_fn)))
    #read_ndvi_img = ndvi_img.read()
    read_pan_img = pan_img.read()
    # print(read_pan_img.shape)
    # (3, 524, 524)
    #comb_img = np.concatenate((read_ndvi_img, read_pan_img), axis=0)
    #comb_img = np.transpose(pan_img, axes=(1,2,0)) #Channel at the end
    #print(fn)
    annotation_im = Image.open(os.path.join(config.base_dir, fn.replace(config.pan_fn,config.annotation_fn)[:-4]+".tif"))
    annotation = np.array(annotation_im)
    # print(annotation.shape)
    # (524, 524)
    weight_im = Image.open(os.path.join(config.base_dir, fn.replace(config.pan_fn,config.weight_fn)[:-4]+".tif"))
    weight = np.array(weight_im)
    # print(weight.shape)
    # (524, 524)
    f = FrameInfo(read_pan_img, annotation, weight)
    #print(f)
    frames.append(f)

In [6]:
training_frames, validation_frames, testing_frames  = split_dataset(frames, config.frames_json, config.patch_dir)
# training_frames = validation_frames = testing_frames  = list(range(len(frames)))
# print(len(training_frames))
# print(len(validation_frames))
# print(len(testing_frames))
annotation_channels = config.input_label_channel + config.input_weight_channel
train_generator = DataGenerator(config.input_image_channel, config.patch_size, training_frames, frames, annotation_channels, augmenter = 'iaa').random_generator(config.BATCH_SIZE, normalize = config.normalize)
val_generator = DataGenerator(config.input_image_channel, config.patch_size, validation_frames, frames, annotation_channels, augmenter= None).random_generator(config.BATCH_SIZE, normalize = config.normalize)
test_generator = DataGenerator(config.input_image_channel, config.patch_size, testing_frames, frames, annotation_channels, augmenter= None).random_generator(config.BATCH_SIZE, normalize = config.normalize)



Reading train-test split from file
training_frames [87, 90, 26, 210, 119, 174, 71, 186, 15, 183, 108, 216, 60, 105, 193, 195, 70, 148, 227, 79, 11, 157, 74, 167, 10, 45, 165, 151, 194, 65, 92, 123, 59, 31, 4, 93, 139, 152, 173, 180, 124, 149, 19, 197, 163, 6, 147, 72, 38, 21, 1, 208, 214, 191, 175, 211, 205, 35, 68, 219, 203, 23, 144, 98, 196, 22, 16, 20, 179, 223, 82, 29, 184, 131, 101, 78, 30, 141, 155, 110, 140, 117, 40, 127, 202, 192, 32, 168, 135, 91, 222, 185, 106, 226, 125, 153, 27, 190, 28, 221, 170, 199, 177, 128, 171, 97, 218, 156, 134, 159, 8, 86, 62, 88, 39, 201, 207, 42, 102, 118, 5, 13, 215, 66, 37, 178, 17, 111, 96, 85, 67, 48, 189, 161, 14, 104, 63, 200, 116, 9, 84, 107, 49, 130, 212]
validation_frames [94, 73, 56, 41, 143, 213, 133, 3, 103, 188, 176, 126, 187, 75, 158, 142, 12, 47, 61, 2, 154, 209, 100, 52, 122, 150, 25, 58, 80, 198, 121, 24, 43, 181, 115, 217, 51]
testing_frames [18, 138, 109, 99, 76, 36, 132, 220, 136, 0, 64, 112, 225, 77, 44, 182, 57, 160, 53, 69, 8

In [8]:
OPTIMIZER = adaDelta
# LOSS = tversky
LOSS = focalTversky
#Only for the name of the model in the very end
OPTIMIZER_NAME = 'AdaDelta'
# LOSS_NAME = 'weightmap_tversky'
LOSS_NAME = 'weightmap_focalTversky'


# Declare the path to the final model
# If you want to retrain an exising model then change the cell where model is declared.
# This path is for storing a model after training.

timestr = time.strftime("%Y%m%d-%H%M")
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '')

print("0")
if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
model_path = os.path.join(config.model_path,'trees_{}_{}_{}_{}_{}.h5'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs,config.input_shape[0]))

In [9]:
config.input_shape

(512, 512, 4)

In [20]:
# Define the model and compile it
model = UNet([config.BATCH_SIZE, *config.input_shape],config.input_label_channel)

model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[accuracy,dice_coef,dice_loss,true_positives,false_positives,true_negatives,false_negatives,sensitivity,specificity,PA,IoU_Pos,IoU_Neg,mIoU,F1_Score])


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 512, 512, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 512, 512, 64  2368        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_39 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_38[0][0]']              
                                )                                                           

 conv2d_51 (Conv2D)             (None, 128, 128, 25  590080      ['conv2d_50[0][0]']              
                                6)                                                                
                                                                                                  
 up_sampling2d_10 (UpSampling2D  (None, 256, 256, 25  0          ['conv2d_51[0][0]']              
 )                              6)                                                                
                                                                                                  
 batch_normalization_22 (BatchN  (None, 256, 256, 25  1024       ['up_sampling2d_10[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 concatenate_10 (Concatenate)   (None, 256, 256, 38  0           ['batch_normalization_22[0][0]', 
          

AttributeError: 'Functional' object has no attribute 'cuda'

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard


checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min', save_weights_only = False)

log_dir = os.path.join(config.model_path,'UNet_{}_{}_{}_{}_{}'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs, config.input_shape[0]))
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

reduceLROnPlat =ReduceLROnPlateau(monitor='val_loss',factor=0.33,patience=5,verbose=1,mode='min',min_delta=0.001,cooldown=4,min_lr=1e-16)
early = EarlyStopping(monitor="val_loss",mode="min",verbose=2,patience=50)


callbacks_list = [checkpoint, tensorboard,early] #reduceLROnPlat is not required with adaDelta

In [ ]:
loss_history = [model.fit(train_generator,
                         steps_per_epoch=config.MAX_TRAIN_STEPS,
                         epochs=config.NB_EPOCHS,
                         validation_data=val_generator,
                         validation_steps=config.VALID_IMG_COUNT,
                         callbacks=callbacks_list,
                         workers=1,
                          )]